In [1]:
from __future__ import division, print_function
%matplotlib inline

from IPython.display import FileLink
import os
import math
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, GlobalAveragePooling2D, GlobalAveragePooling1D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
path = 'data/dogbreeds/'
epochs = 5
batch_size = 16
imsize = 224
transformation_ratio = .005
num_breeds = len(os.listdir(path + 'train'))

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    path + 'train',
    target_size=(imsize, imsize),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    path + 'valid',
    target_size=(imsize, imsize),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = valid_datagen.flow_from_directory(
    path + 'test',
    target_size=(imsize, imsize),
    batch_size=batch_size,
    class_mode=None
)

Found 8221 images belonging to 120 classes.
Found 2001 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [4]:
base_model = Xception(weights='imagenet', include_top=False, pooling='avg')

In [5]:
preds_valid = base_model.predict_generator(valid_generator, math.ceil(valid_generator.samples / batch_size), verbose=1)
np.save('xception_valid', preds_valid)
del preds_valid

125/126 [============================>.] - ETA: 0s

In [8]:
preds_train = np.zeros((train_generator.samples, 2048))

for i in range(math.ceil(train_generator.samples / batch_size)):
    print(i)
    batch = next(train_generator)[0]
    batch_pred = base_model.predict_on_batch(batch)
    
    start = i * batch_size
    end = start + min(batch_size, batch.shape[0])
    preds_train[start:end] = batch_pred
    del batch
    del batch_pred
    
np.save('xception_train', preds_train)
del preds_train

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
df = pd.read_csv('data/dogbreeds/labels.csv')
ids = [id.split('/')[1].split('.')[0] for id in train_generator.filenames]
breeds = sorted(df['breed'].unique())

labels = np.zeros((len(ids), len(breeds)))
for i in range(len(ids)):
    id = ids[i]
    breed = df[df['id'] == id].values[0][1]
    bi = breeds.index(breed)
    labels[i][bi] = 1.0

In [11]:
pred_train = np.load('xception_train.npy')

In [12]:
pred_train.shape

(8221, 2048)

In [20]:
inputs = Input(shape=(2048,))
x = Dense(512, activation='relu')(inputs)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_breeds, activation='softmax')(x)

model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
callbacks_list = [
    EarlyStopping(monitor='val_loss', patience=5)
]

model.fit(
    pred_train, labels,
    epochs=50,
    batch_size=64,
    validation_split=0.2
)

Train on 6576 samples, validate on 1645 samples
Epoch 1/50
6576/6576 [==============================] - 1s - loss: 3.7954 - acc: 0.0970 - val_loss: 14.1656 - val_acc: 0.0000e+00
Epoch 2/50
6576/6576 [==============================] - 1s - loss: 3.7658 - acc: 0.1102 - val_loss: 14.1451 - val_acc: 0.0000e+00
Epoch 3/50
6576/6576 [==============================] - 0s - loss: 3.7975 - acc: 0.1042 - val_loss: 14.1660 - val_acc: 0.0000e+00
Epoch 4/50
6576/6576 [==============================] - 1s - loss: 3.7545 - acc: 0.1089 - val_loss: 14.1724 - val_acc: 0.0000e+00
Epoch 5/50
6576/6576 [==============================] - 1s - loss: 3.7583 - acc: 0.1081 - val_loss: 14.2031 - val_acc: 0.0000e+00
Epoch 6/50
6576/6576 [==============================] - 1s - loss: 3.7633 - acc: 0.1063 - val_loss: 14.3110 - val_acc: 0.0000e+00
Epoch 7/50
6576/6576 [==============================] - 1s - loss: 3.7623 - acc: 0.1134 - val_loss: 14.5075 - val_acc: 0.0000e+00
Epoch 8/50
6576/6576 [====================

KeyboardInterrupt: 

In [28]:
preds = model.predict_generator(test_generator, test_generator.samples, verbose=1)

10357/10357 [==============================] - 297s   


In [29]:
ids = [id[2:-4] for id in test_generator.filenames]
sdf = pd.DataFrame(ids, columns=['id'])
breeds = sorted(train_generator.class_indices.keys())

for index, breed in enumerate(breeds):
    sdf[breed] = np.clip(preds[:,index], 0.01, 0.99)
    
sdf = sdf.sort_values('id').reset_index(drop=True)
sdf.to_csv('pred.csv', index=None)

FileLink('pred.csv')

/home/bill/pred.csv